# Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №2 - Дерево решений
## Гераськин Ярослав

**Общая информация**

**Срок сдачи:** до 1 ноября 2018, 06:00 
**Штраф за опоздание:** -2 балла после 06:00 1 ноября, -4 балла после 06:00 8 ноября, -6 баллов после 06:00 15 ноября

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush
Необходимо в slack создать таск в приватный чат:   
/todo Фамилия Имя *ссылка на гитхаб* @alkhamush   
Пример:   
/todo Ксения Стройкова https://github.com/stroykova/spheremailru/stroykova_hw2.ipynb @alkhamush   

Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (4 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. В комментариях, где написано "Что делает этот блок кода?", ответьте на этот вопрос. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (2 балла)
Добиться скорости работы fit такой, чтобы она была медленнее sklearn не более чем в 10 раз. Скорость проверяем на  wine и Speed Dating Data. Для ускорения используем только numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 3 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw2.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

/home/rickya/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/home/rickya/anaconda3/lib/python3.7/site-packages/pycodestyle.py:113: FutureWarning: Possible nested set at position 1
  EXTRANEOUS_WHITESPACE_REGEX = re.compile(r'[[({] | []}),;:]')


In [2]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None,
                 sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features is None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        def f(x, y):
            return (x ** 2 / y).sum(axis=1)

        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - (f(l_c, l_s) + f(r_c, r_s)) / (l_s[0] + r_s[0])

    def __gini_gain(self, y):
        return 1 - ((np.bincount(y).astype(np.float) / y.size) ** 2).sum()

    def __entropy(self, l_c, l_s, r_c, r_s):
        def f(x, y):
            return ((x / y).log2() * x).nansum(axis=1)

        return -(f(l_c, l_s) + f(r_c, r_s)) / (l_s[0] + r_s[0])

    def __entropy_gain(self, y):
        bc = np.bincount(y).astype(np.float) / y.size
        return -np.nansum(bc * np.log2(bc))

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - (l_c.max(axis=1) + r_c.max(axis=1)) / (l_s[0] + r_s[0])

    def __misclass_gain(self, y):
        return 1 - np.max(np.bincount(y).astype(np.float) / y.size)

    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.sqrt(n_feature))])

    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)

    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #  Что делает этот блок кода?
        # сортирует (x, y) по x
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = self.num_class
        # хотя бы cut_size объектов в каждом из детей
        cut_size = np.int(self.min_samples_split / 2 - 1)

        # Что делает этот блок кода?
        # удаляем по cut_size примеров с каждой стороны и находим индексы,
        # на которых происходит смена класса
        if cut_size == 0:
            splitted_sorted_y = sorted_y
        else:
            splitted_sorted_y = sorted_y[cut_size:-cut_size]
        r_border_ids = np.where(splitted_sorted_y[:-1] !=
                                splitted_sorted_y[1:])[0] + (cut_size + 1)

        if len(r_border_ids) == 0:
            return np.inf, None

        # Что делает этот блок кода?
        # считаем количество элементов в каждом интервале с одним классом
        eq_el_count = r_border_ids - np.append(np.array([cut_size]),
                                               r_border_ids[:-1])
        # 1 в клетке (i, j) значит что в позиции r_border_ids[i] происходит
        # смена класса (был класс j - стал другой)
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]),
                     sorted_y[r_border_ids - 1]] = 1
        # умножаем строки на кол-во элементов в соответствующем интервале
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        # добавляем элементы из части (0, cut_size)
        class_increments[0] = class_increments[0] + \
            np.bincount(sorted_y[:cut_size], minlength=class_number)

        # Что делает этот блок кода?
        # находим кол-во классов в левой и правой части разбиения
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(sorted_y,
                                    minlength=class_number) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        # Что делает этот блок кода?
        # считаем неопределенность для разбиений
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        # выбираем минимальную
        idx = np.argmin(gs)

        # Что делает этот блок кода?
        # возвращаем неопределенносить и порог
        left_el_id = l_sizes[idx][0]
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x, y, node_id, depth, pred_f=-1):
        def make_leaf(y):
            bc = np.bincount(y)
            return (
                self.LEAF_TYPE,
                bc.argmax(),
                bc.astype(np.float) / y.size
            )

        if self.max_depth is not None and depth >= self.max_depth:
            self.tree[node_id] = make_leaf(y)
        if y.shape[0] < self.min_samples_split:
            self.tree[node_id] = make_leaf(y)
        if self.sufficient_share <= np.float(np.bincount(y).max()) / y.size:
            self.tree[node_id] = make_leaf(y)

        ids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, idx], y)
                               for idx in ids])

        best = thresholds[:, 0].argmin()
        t = thresholds[best, 1]

        if t is None:
            self.tree[node_id] = make_leaf(y)
            return

        if self.G_function == self.__gini:
            gain_func = self.__gini_gain
        elif self.G_function == self.__entropy:
            gain_func = self.__entropy_gain
        else:
            gain_func = self.__misclass_gain

        x_l, x_r, y_l, y_r = self.__div_samples(x, y, ids[best], t)
        if x_l.shape[0] == 0 or x_r.shape[0] == 0:
            self.tree[node_id] = make_leaf(y)
            return

        self.tree[node_id] = (
            self.NON_LEAF_TYPE,
            ids[best],
            t
        )

        self.feature_importances_[best] += gain_func(y) * y.size
        self.feature_importances_[best] -= gain_func(y_l) * y_l.size
        self.feature_importances_[best] -= gain_func(y_r) * y_r.size

        self.__fit_node(x_l, y_l, 2 * node_id + 1, depth + 1, pred_f)
        self.__fit_node(x_r, y_r, 2 * node_id + 2, depth + 1, pred_f)

    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.feature_importances_ = np.zeros(x.shape[1], dtype=np.float)
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= self.feature_importances_.sum()

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]

    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])

    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(
    wine.data,
    wine.target,
    test_size=0.1,
    stratify=wine.target,
    random_state=12
)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 2.14 ms, sys: 224 µs, total: 2.37 ms
Wall time: 1.49 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 18.5 ms, sys: 105 µs, total: 18.6 ms
Wall time: 17.1 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.8363636363636364

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [9]:
def get_data():
    df = df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')
    df = df.iloc[:, :97]
    df = df.drop(['id'], axis=1)
    df = df.drop(['idg'], axis=1)
    df = df.drop(['condtn'], axis=1)
    df = df.drop(['round'], axis=1)
    df = df.drop(['position', 'positin1'], axis=1)
    df = df.drop(['order'], axis=1)
    df = df.drop(['partner'], axis=1)
    df = df.drop(['age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int',
                  'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'dec_o', 'attr_o',
                  'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o',
                  'prob_o', 'met_o'], axis=1)
    df.drop_duplicates('iid').age.isnull().sum()
    df = df.dropna(subset=['age'])
    df.loc[:, 'field_cd'] = df.loc[:, 'field_cd'].fillna(0)
    df = df.drop(['field'], axis=1)
    pd.get_dummies(df, columns=['field_cd'], prefix='field_cd', prefix_sep='=')
    df = df.drop(['undergra'], axis=1)
    df.loc[:, 'mn_sat'] = \
        df.loc[:, 'mn_sat'].str.replace(',', '').astype(np.float)
    df.loc[:, 'mn_sat'] = df.mn_sat.fillna(-999)
    df.loc[:, 'tuition'] = \
        df.loc[:, 'tuition'].str.replace(',', '').astype(np.float)
    df.loc[:, 'tuition'] = df.tuition.fillna(-999)
    df = df.dropna(subset=['imprelig', 'imprace'])
    df = df.drop(['from', 'zipcode'], axis=1)
    df.loc[:, 'income'] = \
        df.loc[:, 'income'].str.replace(',', '').astype(np.float)
    df.loc[:, 'income'] = df.loc[:, 'income'].fillna(-999)
    df = df.dropna(subset=['date'])
    df.loc[:, 'career_c'] = df.loc[:, 'career_c'].fillna(0)
    df = df.drop(['career'], axis=1)
    df = df.drop(['sports', 'tvsports', 'exercise', 'dining', 'museums',
                  'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv',
                  'theater', 'movies', 'concerts', 'music', 'shopping',
                  'yoga'], axis=1)
    df = df.drop(['expnum'], axis=1)
    feat = ['iid', 'wave', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1',
            'amb1_1', 'shar1_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    df.loc[:, 'temp_totalsum'] = \
        df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
                   'shar1_1']].sum(axis=1)
    df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1',
               'shar1_1']] = (df.loc[:, ['attr1_1', 'sinc1_1', 'intel1_1',
                                         'fun1_1', 'amb1_1', 'shar1_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    feat = ['iid', 'wave', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1',
            'amb2_1', 'shar2_1']
    temp = df.drop_duplicates(subset=['iid', 'wave']).loc[:, feat]
    temp.loc[:, 'totalsum'] = temp.iloc[:, 2:].sum(axis=1)
    df.loc[:, 'temp_totalsum'] = df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                            'fun2_1', 'amb2_1', 'shar2_1']]\
                                   .sum(axis=1)
    df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1',
               'shar2_1']] = (df.loc[:, ['attr2_1', 'sinc2_1', 'intel2_1',
                                         'fun2_1', 'amb2_1', 'shar2_1']].T /
                              df.loc[:, 'temp_totalsum'].T).T * 100
    df = df.drop(['temp_totalsum'], axis=1)

    for i in [4, 5]:
        feat = ['attr{}_1'.format(i), 'sinc{}_1'.format(i),
                'intel{}_1'.format(i), 'fun{}_1'.format(i),
                'amb{}_1'.format(i), 'shar{}_1'.format(i)]
        if i != 4:
            feat.remove('shar{}_1'.format(i))
        df = df.drop(feat, axis=1)

    df = df.drop(['wave'], axis=1)
    df_male = df.query('gender == 1').drop_duplicates(subset=['iid', 'pid'])\
                .drop(['gender'], axis=1).dropna()
    df_female = df.query('gender == 0').drop_duplicates(subset=['iid'])\
                  .drop(['gender', 'match', 'int_corr', 'samerace'], axis=1)\
                  .dropna()

    df_female.columns = df_female.columns + '_f'
    df_female = df_female.drop(['pid_f'], axis=1)
    df_pair = df_male.join(df_female.set_index('iid_f'), on='pid', how='inner')
    df_pair = df_pair.drop(['iid', 'pid'], axis=1)
    X = df_pair.iloc[:, 1:].values
    y = df_pair.iloc[:, 0].values
    features_names = df_pair.columns[1:]

    return X, y, features_names

In [10]:
X, y, features_names = get_data()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=12
)

## Проверка скорости работы на Speed Dating Data 

In [11]:
%time clf.fit(X_train, y_train)

CPU times: user 387 ms, sys: 4.21 ms, total: 391 ms
Wall time: 97.7 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
%time my_clf.fit(X_train, y_train)

CPU times: user 1.31 s, sys: 7.04 ms, total: 1.32 s
Wall time: 1.25 s


## Проверка качества работы на Speed Dating Data

In [13]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.5306666666666666

In [14]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.5679579565056181

## Задание 3

In [15]:
def show_importances(names, clf, n=10):
    ids = clf.feature_importances_.argsort()[-1:-n - 1:-1]
    for idx in ids:
        print(
            'name {}, value {:0.4f}'.format(
                names[idx],
                clf.feature_importances_[idx]
            )
        )

In [16]:
show_importances(features_names, clf)

name int_corr, value 0.0798
name sinc1_1, value 0.0297
name shar2_1, value 0.0276
name career_c_f, value 0.0260
name amb1_1_f, value 0.0248
name amb3_1_f, value 0.0247
name fun2_1_f, value 0.0214
name income_f, value 0.0213
name sinc2_1_f, value 0.0213
name fun1_1_f, value 0.0210


In [17]:
show_importances(features_names, my_clf)

name int_corr, value 0.1290
name age_f, value 0.0502
name attr1_1, value 0.0456
name intel1_1_f, value 0.0417
name imprelig_f, value 0.0400
name age, value 0.0377
name attr2_1_f, value 0.0374
name tuition_f, value 0.0370
name field_cd_f, value 0.0359
name race, value 0.0312


## Задание 4

In [18]:
clf = RandomForestClassifier(random_state=12)
n_iter = 1000

params = {
    'max_depth': [None] + list(range(1, 11)),
    'n_estimators': range(10, 51),
    'criterion': ['gini', 'entropy'],
    'min_samples_split': range(2, 11)
}

rs = RandomizedSearchCV(
    clf,
    param_distributions=params,
    n_iter=n_iter,
    n_jobs=-1
)

In [19]:
%%time
rs.fit(X, y)
print(rs.best_params_)

{'n_estimators': 20, 'min_samples_split': 5, 'max_depth': 4, 'criterion': 'entropy'}
CPU times: user 12.7 s, sys: 250 ms, total: 13 s
Wall time: 1min 43s
